In [17]:
import torch
import os
from ultralytics import YOLO
import cv2
import fnmatch
import csv
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
from multiprocessing import Manager
import tempfile

In [25]:
%cd Brands/

/home/hawga970/Desktop/Brands


In [26]:
brands_model = YOLO('runs/detect/train/weights/best.pt')

In [27]:
%cd '~'
%cd 'Desktop'

/home/hawga970
/home/hawga970/Desktop


In [28]:
%cd Alcohol/

/home/hawga970/Desktop/Alcohol


In [29]:
alc_model = YOLO('runs/detect/ALCOHOL_100/weights/best.pt')

In [30]:
%cd '~'

/home/hawga970


In [31]:
ALC_outputFilename = "Desktop/Frames/Alcohol/FINAL.csv" 
if os.path.exists(ALC_outputFilename):
    print(f"Predictions file already exists in the \'{ALC_outputFilename}\'.")
    print(f"Please delete the existing {ALC_outputFilename} to rerun the code.")

BRANDS_outputFilename = "Desktop/Frames/Brands/FINAL.csv" 
if os.path.exists(BRANDS_outputFilename):
    print(f"Predictions file already exists in the \'{BRANDS_outputFilename}\'.")
    print(f"Please delete the existing {BRANDS_outputFilename} to rerun the code.")

Predictions file already exists in the 'Desktop/Frames/Alcohol/FINAL.csv'.
Please delete the existing Desktop/Frames/Alcohol/FINAL.csv to rerun the code.
Predictions file already exists in the 'Desktop/Frames/Brands/FINAL.csv'.
Please delete the existing Desktop/Frames/Brands/FINAL.csv to rerun the code.


In [32]:
def frames_equal(frame1, frame2):
    gray_frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray_frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    diff = cv2.absdiff(gray_frame1, gray_frame2)
    _, diff_threshold = cv2.threshold(diff, 50, 255, cv2.THRESH_BINARY)
    diff_count = cv2.countNonZero(diff_threshold)
    
    return diff_count < 2000


def overall_grayscale_intensity(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    intensity = cv2.mean(gray_frame)[0]
    
    return intensity

In [33]:
def process_results(results):
    ret = []
    conf = results[0].boxes.conf
    conf = conf.detach().cpu().numpy().tolist()
    for r in results:
        res = []
        nums = r.boxes.cls.numpy()
        names = r.names
        for i in nums:
            res.append(names[i])
        print(res)
    if not res:
        return 0
    else: 
        locations = results[0].boxes.xywh
        locations = locations.detach().cpu().numpy().tolist()
        locations = [':'.join(f'{round(num, 2)}' for num in sublist) for sublist in locations]
        for i in range(len(res)):
            ret.append([conf[i], res[i], locations[i]])
    return ret

In [34]:
def extract_frames(video_path, output_path):
    alc_rows = []
    brands_rows = []
    video_capture = cv2.VideoCapture(video_path)
    
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_counter = 0
    prev_frame = None  
    last_counter = 0
    last=True
    
    while frame_counter < total_frames:
        frame_pos = int(frame_counter)
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)

        ret, frame = video_capture.read()

        if ret:
            height, width = frame.shape[:2]
            if width > height:
                scale_factor = 800 / width
            else:
                scale_factor = 800 / height
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            frame = cv2.resize(frame, (new_width, new_height))

        if not ret:
            break

        if prev_frame is None or not frames_equal(frame, prev_frame) or last_counter>=7:
            if last_counter>=7 and not frames_equal(frame, prev_frame): 
                last = False
            else: 
                if last_counter>=7: last = True
            items = [frame] if last else [frame, prev_frame] 
            timestamp = [frame_counter/fps] if last else [frame_counter/fps, (frame_counter-(fps/4))/fps]
            for item, ts in zip(items, timestamp):
                if overall_grayscale_intensity(item) > 5:
                    minutes, seconds = divmod(ts, 60)
                    hours, minutes = divmod(minutes, 60)
                    time_str = f"{int(hours):02d}_{int(minutes):02d}_{seconds:05.2f}"
                    output_frame_path = f"{os.path.splitext(os.path.basename(video_path))[0]}_{time_str}.jpg"

                    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
                        alc_future = executor.submit(process_results, alc_model([item], conf=0.2))
                        brands_future = executor.submit(process_results, brands_model([item], conf=0.2))

                        proc_alc = alc_future.result()
                        proc_brands = brands_future.result()
                    
                    video_path_parts = video_path.split('/')
                            
                    if proc_alc == 0:
                        print(f"Nothing Here: {output_frame_path}")
                    else: 
                        for i in range(len(proc_alc)):
                            alc_rows.append(f"\n{video_path_parts[1]}, {video_path_parts[2]}, {video_path_parts[3].replace(",", "").replace(" ", "_")}, {video_path_parts[4]},{time_str},{proc_alc[i][0]}, {proc_alc[i][1]}, {proc_alc[i][2]}")
                        print(f"Frame saved: {output_frame_path}")

                    if proc_brands == 0:
                        print(f"Nothing Here: {output_frame_path}")
                    else: 
                        for i in range(len(proc_brands)):
                            brands_rows.append(f"\n{video_path_parts[1]}, {video_path_parts[2]}, {video_path_parts[3].replace(",", "").replace(" ", "_")}, {video_path_parts[4]},{time_str},{proc_brands[i][0]}, {proc_brands[i][1]}, {proc_brands[i][2]}")
                        print(f"Frame saved: {output_frame_path}")
                
                last_counter=0
                last=True
            
        else:
            last_counter = last_counter+1
            last=False
        prev_frame = frame

        frame_counter += fps / 4
    
    video_capture.release()
    print(f"Frames extraction complete for video {video_path}")
    return alc_rows, brands_rows

In [37]:
def process_video(args):
    video_path, output_path, alc_tmp_file, brands_tmp_file = args
    alc_dataframe, brands_dataframe = extract_frames(video_path, output_path)
    
    cleaned_alc_rows = [row.strip() for row in alc_dataframe]
    cleaned_brands_rows = [row.strip() for row in brands_dataframe]

    if cleaned_alc_rows:
        with open(alc_tmp_file, mode="a") as alc_file:
            alc_file.write("\n".join(cleaned_alc_rows) + "\n")

    if cleaned_brands_rows:
        with open(brands_tmp_file, mode="a") as brands_file:
            brands_file.write("\n".join(cleaned_brands_rows) + "\n")

def extract_videos_from_folder(folder_path, output_path, ALC_outputFilename, BRANDS_outputFilename):
    first_videos = []
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if fnmatch.fnmatch(filename, '*.mp4'):
                if "_avo_" not in filename:
                    name = os.path.join(root, filename)
                    if 'analytics' not in name.lower() and 'digital' not in name.lower():
                        first_videos.append(name)
                    break
    if not first_videos:
        print("No .mp4 files found in any end folder.")
        return

    """with open(ALC_outputFilename, mode="w") as alc_file:
        alc_file.write("Student,Device,DateTime,VideoFile,Timestamp,Confidence,Prediction,Location\n")

    with open(BRANDS_outputFilename, mode="w") as brands_file:
        brands_file.write("Student,Device,DateTime,VideoFile,Timestamp,Confidence,Prediction,Location\n")"""

    with ProcessPoolExecutor(max_workers=15) as executor:
        executor.map(process_video, [(video, output_path, ALC_outputFilename, BRANDS_outputFilename) for video in first_videos])

In [ ]:
folder_path = "STUDENT_DATA/"
output_path = "Frames/test/"

extract_videos_from_folder(folder_path, output_path, ALC_outputFilename, BRANDS_outputFilename)

STUDENT_DATA/SWIS69/Laptop/Sep 15, 2023 1116 AM Auckland, Wellington/GMT20230914-231653_Recording_1298x730.mp4
STUDENT_DATA/SWIS69/Laptop/Sep 14, 2023 0223 PM Auckland, Wellington/GMT20230914-022349_Recording_as_1298x730.mp4
STUDENT_DATA/SWIS69/Laptop/Sep 15, 2023 0909 AM Auckland, Wellington/GMT20230914-210951_Recording_1298x730.mp4
STUDENT_DATA/PAPA63/Laptop/Aug 3, 2023 0319 PM Auckland, Wellington/GMT20230803-031943_Recording_as_1366x768.mp4
STUDENT_DATA/EBIS135/Phone/Jun 12, 2024 0911 AM/GMT20240611-211154_Recording_1280x720.mp4
STUDENT_DATA/EBIS135/Phone/Jun 12, 2024 0908 AM/GMT20240611-210853_Recording_1280x960.mp4
STUDENT_DATA/EBIS135/Phone/Jun 11, 2024 0952 AM/GMT20240610-215221_Recording_1280x960.mp4
STUDENT_DATA/EBIS135/Phone/Jun 15, 2024 0406 PM/GMT20240615-040606_Recording_1280x720.mp4
STUDENT_DATA/EBIS135/Phone/Jun 15, 2024 0435 PM/GMT20240615-043835_Recording_0x0.mp4
STUDENT_DATA/EBIS135/Phone/Jun 17, 2024 0936 AM/GMT20240616-213646_Recording_1280x960.mp4
STUDENT_DATA/EBI